<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#ETL-Pipeline-Preparation" data-toc-modified-id="ETL-Pipeline-Preparation-1">ETL Pipeline Preparation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#1.-Import-libraries-and-load-datasets." data-toc-modified-id="1.-Import-libraries-and-load-datasets.-1.0.1">1. Import libraries and load datasets.</a></span></li><li><span><a href="#2.-Merge-datasets." data-toc-modified-id="2.-Merge-datasets.-1.0.2">2. Merge datasets.</a></span></li><li><span><a href="#3.-Split-categories-into-separate-category-columns." data-toc-modified-id="3.-Split-categories-into-separate-category-columns.-1.0.3">3. Split <code>categories</code> into separate category columns.</a></span></li><li><span><a href="#4.-Convert-category-values-to-just-numbers-0-or-1." data-toc-modified-id="4.-Convert-category-values-to-just-numbers-0-or-1.-1.0.4">4. Convert category values to just numbers 0 or 1.</a></span></li><li><span><a href="#5.-Replace-categories-column-in-df-with-new-category-columns." data-toc-modified-id="5.-Replace-categories-column-in-df-with-new-category-columns.-1.0.5">5. Replace <code>categories</code> column in <code>df</code> with new category columns.</a></span></li><li><span><a href="#6.-Remove-duplicates." data-toc-modified-id="6.-Remove-duplicates.-1.0.6">6. Remove duplicates.</a></span></li><li><span><a href="#7.-Save-the-clean-dataset-into-an-sqlite-database." data-toc-modified-id="7.-Save-the-clean-dataset-into-an-sqlite-database.-1.0.7">7. Save the clean dataset into an sqlite database.</a></span></li><li><span><a href="#8.-Use-this-notebook-to-complete-etl_pipeline.py" data-toc-modified-id="8.-Use-this-notebook-to-complete-etl_pipeline.py-1.0.8">8. Use this notebook to complete <code>etl_pipeline.py</code></a></span></li></ul></li></ul></li></ul></div>

# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
import os

In [2]:
# load messages dataset
messages = pd.read_csv('../data/disaster_messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
messages.shape

(26248, 4)

In [4]:
# load categories dataset
categories = pd.read_csv('../data/disaster_categories.csv', sep=',')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [5]:
# merge datasets
df = messages.merge(categories, how='inner', on='id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [6]:
# create a dataframe of the 36 individual category columns
categories = df.categories.str.split(';', expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [7]:
# select the first row of the categories dataframe
row = categories[:1]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.apply(lambda x: x.values[0].split('-')[0]).values
print(category_colnames)

['related' 'request' 'offer' 'aid_related' 'medical_help'
 'medical_products' 'search_and_rescue' 'security' 'military'
 'child_alone' 'water' 'food' 'shelter' 'clothing' 'money'
 'missing_people' 'refugees' 'death' 'other_aid' 'infrastructure_related'
 'transport' 'buildings' 'electricity' 'tools' 'hospitals' 'shops'
 'aid_centers' 'other_infrastructure' 'weather_related' 'floods' 'storm'
 'fire' 'earthquake' 'cold' 'other_weather' 'direct_report']


In [8]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [9]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str[-1]
    
    # convert column from string to numeric
    categories[column] = categories[column].astype(int)
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [10]:
# drop the original categories column from `df`
df.drop('categories', axis=1, inplace=True)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [11]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [12]:
# check number of duplicates
df.duplicated().sum()

170

In [13]:
# drop duplicates
df.drop_duplicates(inplace=True)

In [14]:
# check number of duplicates
df.duplicated().sum()

0

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [15]:
from sqlalchemy import create_engine

In [16]:
df.shape

(26216, 40)

In [17]:
df[df.original.isna()]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7483,8365,NOTES: It mark as not enough information,NaN,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9956,11186,My thoughts and prayers go out to all the live...,NaN,social,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9957,11188,I m sorry for the poor people in Haiti tonight...,NaN,social,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9958,11189,RT selenagomez UNICEF has just announced an em...,NaN,social,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
9959,11192,lilithia yes 5.2 magnitude earthquake hit mani...,NaN,social,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9960,11193,RT TheNewsBlotter RT caribnews On Call Interna...,NaN,social,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
9961,11195,Most Eureka homeowners won&#39 t collect on qu...,NaN,social,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9962,11196,Haiti hit by largest earthquake in over 200 ye...,NaN,social,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9963,11197,RT pinkelephantpun Earthquake Relief Donate ht...,NaN,social,1,1,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
9964,11198,Praying that today s earthquake in #Haiti mini...,NaN,social,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


In [18]:
df.fillna('Already English', inplace=True)

Issue https://github.com/pandas-dev/pandas/issues/21103 

After further testing the initial assumption seems correct and is based on the SQLITE_MAX_VARIABLE_NUMBER limit of 999. The max allowable chunksize associated with pd.df_to_sql is given by chunksize=999//(cols+1)

Thus we have to iterate 

In [19]:
pd.__version__

'0.23.0'

In [20]:
db_name = 'Message'
db_file = '../db/DisasterResponse.db'
engine = create_engine('sqlite:///'+db_file)
with engine.begin() as conn:
    for start in range(0, df.shape[0], 20):
        if start%5000 == 0:
            print(start)
        end = start+20
        if start+20 > df.shape[0]:
            end = df.shape[0]
        df[start:end].to_sql(name=db_name, con=conn, index=True, if_exists='append')
print('Wrote %d rows to %s in %s'%(df.shape[0], db_name, os.path.abspath(db_file)))

0
5000
10000
15000
20000
25000
Wrote 26216 rows to Message in /Users/louwrenslabuschagne/Documents/gitProjects/disaster-response-pipeline/db/DisasterResponse.db


### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.